# Basicamente estamos haciendo un label para los sentimientos y luego un knn para los valores nan, despues lo devolvemos todo a str,todo esto lo he hecho para poder agregarlo al codigo de los ejemplos (CODE NEIGHBORS)

In [1]:
import pandas as pd 
import numpy as np
import pandas as pd
import pickle

# Encoders
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

# Data Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Feature Importance
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_processed_ML.csv')
df_sentiment = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_sentiment_mean.csv')
#Abro el csv de df_final_cleaned para unir sentiment y aprocesado y crear un csv para sql
df_cleaned = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_final_cleaned.csv')

In [3]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2456 entries, 0 to 2455
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Url                          2456 non-null   object 
 1   mean_sentiment               2456 non-null   float64
 2   sentiment_label_transformed  2456 non-null   object 
dtypes: float64(1), object(2)
memory usage: 57.7+ KB


unimos df

In [4]:
df['sentiment_label_transformed'] = df_sentiment['sentiment_label_transformed']

Reemplazamos NaN por una categoría missing, se hace para que una vez hecho el label no hayan problemas

In [5]:
df['sentiment_label_transformed'] = df['sentiment_label_transformed'].fillna('missing')

Usamos el Label

In [6]:
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment_label_transformed'])
df.head()

,prices_per_night,ratings,cleaning_fee,dormitorios,camas,baños,maximum_guests,check_in_hour,check_out_hour,total_hours_checkin,...,exterior,internet y oficina,para familias,privacidad y seguridad,seguridad en el hogar,servicios,habitacion,alojamiento entero,sentiment_label_transformed,sentiment_encoded
0,115.0,0.00,0.0,1.0,1.0,1.0,2.0,900.0,720.0,9.0,...,2.0,1.0,0.0,0.0,1.0,4.0,0.0,0.0,Positivo,2
1,46.0,0.00,15.0,1.0,1.0,0.5,1.0,1020.0,660.0,7.0,...,0.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0,Positivo,2
2,47.0,4.66,0.0,1.0,1.0,0.5,1.0,900.0,720.0,9.0,...,1.0,1.0,0.0,1.0,0.0,5.0,1.0,0.0,Positivo,2
3,100.0,4.89,35.0,1.0,1.0,1.0,1.0,960.0,720.0,8.0,...,3.0,2.0,2.0,0.0,5.0,4.0,0.0,0.0,Positivo,2
4,33.0,4.40,0.0,1.0,1.0,0.5,1.0,900.0,660.0,9.0,...,1.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,Positivo,2


Aplicamos el KNN solo para esa columna

In [7]:
imputer = KNNImputer(n_neighbors=2)  # Puedes ajustar el número de vecinos
df['sentiment_imputed'] = imputer.fit_transform(df[['sentiment_encoded']])
df.head()

,prices_per_night,ratings,cleaning_fee,dormitorios,camas,baños,maximum_guests,check_in_hour,check_out_hour,total_hours_checkin,...,internet y oficina,para familias,privacidad y seguridad,seguridad en el hogar,servicios,habitacion,alojamiento entero,sentiment_label_transformed,sentiment_encoded,sentiment_imputed
0,115.0,0.00,0.0,1.0,1.0,1.0,2.0,900.0,720.0,9.0,...,1.0,0.0,0.0,1.0,4.0,0.0,0.0,Positivo,2,2.0
1,46.0,0.00,15.0,1.0,1.0,0.5,1.0,1020.0,660.0,7.0,...,2.0,0.0,2.0,0.0,1.0,1.0,0.0,Positivo,2,2.0
2,47.0,4.66,0.0,1.0,1.0,0.5,1.0,900.0,720.0,9.0,...,1.0,0.0,1.0,0.0,5.0,1.0,0.0,Positivo,2,2.0
3,100.0,4.89,35.0,1.0,1.0,1.0,1.0,960.0,720.0,8.0,...,2.0,2.0,0.0,5.0,4.0,0.0,0.0,Positivo,2,2.0
4,33.0,4.40,0.0,1.0,1.0,0.5,1.0,900.0,660.0,9.0,...,1.0,0.0,0.0,3.0,1.0,1.0,0.0,Positivo,2,2.0


Reconvertimos los valores númericos a las categorias originales

In [8]:
# Paso 4: Reconvertir los valores numéricos imputados a las categorías originales
df['sentiment_imputed'] = df['sentiment_imputed'].round().astype(int)
df['sentiment_final'] = label_encoder.inverse_transform(df['sentiment_imputed'])
df

,prices_per_night,ratings,cleaning_fee,dormitorios,camas,baños,maximum_guests,check_in_hour,check_out_hour,total_hours_checkin,...,para familias,privacidad y seguridad,seguridad en el hogar,servicios,habitacion,alojamiento entero,sentiment_label_transformed,sentiment_encoded,sentiment_imputed,sentiment_final
0,115.0,0.00,0.0,1.0,1.0,1.0,2.0,900.0,720.0,9.0,...,0.0,0.0,1.0,4.0,0.0,0.0,Positivo,2,2,Positivo
1,46.0,0.00,15.0,1.0,1.0,0.5,1.0,1020.0,660.0,7.0,...,0.0,2.0,0.0,1.0,1.0,0.0,Positivo,2,2,Positivo
2,47.0,4.66,0.0,1.0,1.0,0.5,1.0,900.0,720.0,9.0,...,0.0,1.0,0.0,5.0,1.0,0.0,Positivo,2,2,Positivo
3,100.0,4.89,35.0,1.0,1.0,1.0,1.0,960.0,720.0,8.0,...,2.0,0.0,5.0,4.0,0.0,0.0,Positivo,2,2,Positivo
4,33.0,4.40,0.0,1.0,1.0,0.5,1.0,900.0,660.0,9.0,...,0.0,0.0,3.0,1.0,1.0,0.0,Positivo,2,2,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,55.0,4.74,10.0,1.0,1.0,0.5,3.0,900.0,660.0,9.0,...,0.0,3.0,0.0,1.0,1.0,0.0,missing,3,3,missing
2493,60.0,4.78,0.0,1.0,1.0,0.5,2.0,900.0,600.0,9.0,...,0.0,1.0,0.0,1.0,1.0,0.0,missing,3,3,missing
2494,104.0,4.96,0.0,2.0,3.0,2.0,4.0,900.0,720.0,9.0,...,0.0,0.0,2.0,3.0,0.0,0.0,missing,3,3,missing
2495,120.0,4.83,50.0,1.0,1.0,1.0,2.0,900.0,660.0,9.0,...,0.0,0.0,2.0,1.0,0.0,0.0,missing,3,3,missing


In [9]:
df

,prices_per_night,ratings,cleaning_fee,dormitorios,camas,baños,maximum_guests,check_in_hour,check_out_hour,total_hours_checkin,...,para familias,privacidad y seguridad,seguridad en el hogar,servicios,habitacion,alojamiento entero,sentiment_label_transformed,sentiment_encoded,sentiment_imputed,sentiment_final
0,115.0,0.00,0.0,1.0,1.0,1.0,2.0,900.0,720.0,9.0,...,0.0,0.0,1.0,4.0,0.0,0.0,Positivo,2,2,Positivo
1,46.0,0.00,15.0,1.0,1.0,0.5,1.0,1020.0,660.0,7.0,...,0.0,2.0,0.0,1.0,1.0,0.0,Positivo,2,2,Positivo
2,47.0,4.66,0.0,1.0,1.0,0.5,1.0,900.0,720.0,9.0,...,0.0,1.0,0.0,5.0,1.0,0.0,Positivo,2,2,Positivo
3,100.0,4.89,35.0,1.0,1.0,1.0,1.0,960.0,720.0,8.0,...,2.0,0.0,5.0,4.0,0.0,0.0,Positivo,2,2,Positivo
4,33.0,4.40,0.0,1.0,1.0,0.5,1.0,900.0,660.0,9.0,...,0.0,0.0,3.0,1.0,1.0,0.0,Positivo,2,2,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,55.0,4.74,10.0,1.0,1.0,0.5,3.0,900.0,660.0,9.0,...,0.0,3.0,0.0,1.0,1.0,0.0,missing,3,3,missing
2493,60.0,4.78,0.0,1.0,1.0,0.5,2.0,900.0,600.0,9.0,...,0.0,1.0,0.0,1.0,1.0,0.0,missing,3,3,missing
2494,104.0,4.96,0.0,2.0,3.0,2.0,4.0,900.0,720.0,9.0,...,0.0,0.0,2.0,3.0,0.0,0.0,missing,3,3,missing
2495,120.0,4.83,50.0,1.0,1.0,1.0,2.0,900.0,660.0,9.0,...,0.0,0.0,2.0,1.0,0.0,0.0,missing,3,3,missing


Drop de las columnas que no nos interesan (son como columnas intermedias)

In [10]:
df = df.drop(columns=['sentiment_label_transformed', 'sentiment_encoded', 'sentiment_imputed'])

Cambiamos los missing por neutro

In [11]:
df['sentiment_final'] = df['sentiment_final'].replace('missing', 'neutro')

In [12]:
df

,prices_per_night,ratings,cleaning_fee,dormitorios,camas,baños,maximum_guests,check_in_hour,check_out_hour,total_hours_checkin,...,entretenimiento,exterior,internet y oficina,para familias,privacidad y seguridad,seguridad en el hogar,servicios,habitacion,alojamiento entero,sentiment_final
0,115.0,0.00,0.0,1.0,1.0,1.0,2.0,900.0,720.0,9.0,...,1.0,2.0,1.0,0.0,0.0,1.0,4.0,0.0,0.0,Positivo
1,46.0,0.00,15.0,1.0,1.0,0.5,1.0,1020.0,660.0,7.0,...,0.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0,Positivo
2,47.0,4.66,0.0,1.0,1.0,0.5,1.0,900.0,720.0,9.0,...,1.0,1.0,1.0,0.0,1.0,0.0,5.0,1.0,0.0,Positivo
3,100.0,4.89,35.0,1.0,1.0,1.0,1.0,960.0,720.0,8.0,...,10.0,3.0,2.0,2.0,0.0,5.0,4.0,0.0,0.0,Positivo
4,33.0,4.40,0.0,1.0,1.0,0.5,1.0,900.0,660.0,9.0,...,0.0,1.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,55.0,4.74,10.0,1.0,1.0,0.5,3.0,900.0,660.0,9.0,...,1.0,0.0,2.0,0.0,3.0,0.0,1.0,1.0,0.0,neutro
2493,60.0,4.78,0.0,1.0,1.0,0.5,2.0,900.0,600.0,9.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,neutro
2494,104.0,4.96,0.0,2.0,3.0,2.0,4.0,900.0,720.0,9.0,...,1.0,0.0,2.0,0.0,0.0,2.0,3.0,0.0,0.0,neutro
2495,120.0,4.83,50.0,1.0,1.0,1.0,2.0,900.0,660.0,9.0,...,3.0,3.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,neutro


Creamos el csv que usaremos en el CODE NEIGHBORS

In [18]:
df.to_csv('df_sentiment_complet.csv', index = False)

In [14]:
df.isna().sum()

prices_per_night                   0
ratings                            0
cleaning_fee                       0
dormitorios                        0
camas                              0
baños                              0
maximum_guests                     0
check_in_hour                      0
check_out_hour                     0
total_hours_checkin                0
log_num_reviews                    0
aparcamiento e instalaciones       0
baño                               0
calefacción y refrigeración        0
características de la ubicación    0
cocina y comedor                   0
dormitorio y lavandería            0
entretenimiento                    0
exterior                           0
internet y oficina                 0
para familias                      0
privacidad y seguridad             0
seguridad en el hogar              0
servicios                          0
habitacion                         0
alojamiento entero                 0
sentiment_final                    0
d

Para SQL uno solo los sentimientos con el df_cleaned

In [15]:
df_merged = df_cleaned.merge(df, left_index=True, right_index=True, how='inner')

In [23]:
df_merged['sentiment_final'].value_counts()

sentiment_final
Positivo    2438
neutro        41
Neutral       16
Negativo       2
Name: count, dtype: int64

In [24]:
df_merged['sentiment_final'].isna().sum()

0

In [19]:
df_merged.to_csv('df_cleaned_sentiment_sql.csv', index = False)